In [ ]:
import shutil
import sys
import os.path
import pandas as pd

if not shutil.which("pyomo"):
    !pip install -q pyomo
assert(shutil.which("pyomo"))

if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk
        except:
            pass
assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

from pyomo.environ import *

# **Optimización discreta**
En este segundo bloque del taller aprenderás a:
- Declarar modelos de optimización de forma abstracta
- Definir variables discretas y restricciones lógicas
- Aplicar reformulaciones tipo $M$ grande y envolvente convexa

# **Ejemplo 1:** problema de ubicación de almacenes

La ubicación de almacenes es un problema típico de optimización discreta. 

![](https://www.researchgate.net/publication/354780932/figure/fig1/AS:1071172368232449@1632398802931/Decision-stages-of-the-facility-location-problem-with-user-preferences.png)

Tu empresa está llevando a cabo un plan de expansión y, para abastecer a las nuevas tiendas, es necesario construir nuevos almacenes. Las reglas a tener en cuenta son las siguientes:
- Cada almacén tiene un coste de explotación fijo
- Cada almacen una capacidad máxima que especifica el número de tiendas que puede atender
- Cada tienda debe ser abastecida exactamente por un almacén 
- El coste de abastecer a una tienda depende del almacén seleccionado

Tu tarea consiste en elegir qué almacenes construir y qué almacén asignar a cada tienda para minimizar el coste total, es decir, la suma de los costes fijos y de suministro. Para ello, plantea el problema de optimización y resuélvelo con Pyomo.

## Implementación del modelo

Este problema se va a modelar como un `AbstractModel`, por lo que los datos se cargarán a posteriori una vez definido el modelo.

Al utilizar modelos abstractos, los sets, variables y parámetros se declaran de forma genérica, es decir, sin especificar qué elementos o qué valores van a tomar. De esta manera, puede utilizarse como patrón y aplicarlo a múltiples ejemplos o casos de estudio diferentes.

In [ ]:
model  = ...

# Sets
model.C = ...
model.W = ...

# Variables continuas
model.x = ...

# Variables binarias
model.y = ...

# Parámetros
model.cost = ...
model.P = ...

La función objetivo se define utilizando una función que toma como argumento de entrada la variable que hemos asociado al objeto de tipo *AbstractModel* y devuelven el valor de la función objetivo.

In [ ]:
model.obj = ...

Generalmente, las restricciones se especifican utilizando igualdades (==) o desigualdades (<=, =>) que se generan utilizando "reglas" o funciones. A diferencia de la función objetivo, las restricciones pueden repetirse para varios elementos de un mismo set. En estos casos, las reglas toman como argumento de entrada el set del que dependen.

In [ ]:
model.one_per_cust = ...

In [ ]:
model.num_warehouses = ...

In [ ]:
model.warehouse_active = ...

## Carga de datos

Antes de poder resolver el problema de optimización, hay que crear una instancia concreta del modelo especificando datos empleando la función `create_instance`.

```python
concrete_model = abstract_model.create_instance(data)
```

Esta función toma dos argumentos de entrada (un `AbstractModel` y los datos del problema) y devuelve un `ConcreteModel` con la misma estructura que el problema abstracto que se ha definido, pero incluyendo los datos correspondientes.

En la siguiente tabla aparecen los costes de transporte entre las ciudadaes y los almacenes.
<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|    | NYC | LA | Chicago | Houston |
| :--- | :---: | :---: | :---: | :---: |
| Harlingen | 1956 | 1606 | 1410 | 330 |
| Memphis | 1096 | 1792 | 531 | 330 |
| Ashland | 485 | 2322 | 324 | 1236 |
</div>

📝 **Tarea:** expresa la tabla anterior como un diccionario.

💡 **Pista:** las *keys* del diccionario son tuplas (almacén, ciudad) y los *values*, el coste.
```python
coste = {('Harlingen','NYC'): 1956,
         ('Harlingen','LA'): 1606,
         ...
         }
```

In [ ]:
coste = {
        }

Los datos pueden especificarse de [diversas maneras](https://pyomo.readthedocs.io/en/6.8.0/working_abstractmodels/data/index.html). Para problemas de pequeño tamaño, lo más sencillo es utilizar un diccionario como se muestra a continuación.

In [ ]:
data = {
        None: 
            {
                'W': ['Harlingen', 'Memphis', 'Ashland'], # Almacenes
                'C': ['NYC', 'LA', 'Chicago', 'Houston'], # Ciudades
                'P': {None: 2} , # Número máximo de tiedas que puede abastecer un almacén
                'cost': coste # Costes de suministro
            }
        }

instance = model.create_instance(data)

In [ ]:
model.is_constructed()
instance.is_constructed()

## Selección y llamada al solver
Utiliza el paquete [GLPK (GNU Linear Programming Kit)](https://www.gnu.org/software/glpk/).

In [ ]:
optimizer = ...
results = ...

## Inspeccionar la solución

In [ ]:
print('Distribución óptima:')
instance.x.pprint()
print('Coste total:', instance.obj(),'  UM')

In [ ]:
print(results.solver.status)
print(results.solver.termination_condition)

## Análisis de sensibilidad
Prueba a cambiar alguno de los parámetros del modelo y ver cómo afecta a la solución. Por ejemplo, ¿qué ocurre cuando cada almacén abastece a una única tienda?

In [ ]:
instance.P = 1

In [ ]:
optimizer = SolverFactory('glpk')
results = optimizer.solve(instance)

In [ ]:
print('Distribución óptima')
instance.x.pprint()
print('Coste total:', instance.obj(),'  USD')

# **Ejemplo 2:** Formulación de mezclas
Te han encargado la tarea de producir un nuevo complejo multivitamínico. Teniendo en cuenta una serie de especificaciones de calidad y de condiciones, determina la mezcla y/o combinación óptima de los diferentes compuestos que resulte en el menor coste posible.

## Datos
- Cantidad máxima de vitamina A: 0.4
- Cantidad mínima de vitamina B: 0.2
- Compuestos A y B son incompatibles
- Coste y composición de cada compuesto:

<table style="margin:auto; border-collapse: collapse;">
  <tr style="border-bottom:1px solid #999;">
    <th></th><th>A</th><th>B</th><th>C</th>
  </tr>
  <tr>
    <td><strong>Coste</strong></td><td>2</td><td>2</td><td>5</td>
  </tr>
  <tr>
    <td><strong>VitA</strong></td><td>0.5</td><td>0.4</td><td>0.3</td>
  </tr>
  <tr>
    <td><strong>VitB</strong></td><td>0.2</td><td>0.1</td><td>0.3</td>
  </tr>
</table>

In [ ]:
data = {
        'Coste':{'A': 2. , 'B': 2.  , 'C': 5.  }, 
        'VitA': {'A': .5 ,'B': .4, 'C': .3 }, 
        'VitB': {'A': .2 , 'B': .1 , 'C': .3 }
        }

Implementación del modelo

In [ ]:
model  = ConcreteModel()

# Sets
model.C = ...
model.C_incompat = ...

# Variables
model.x = ...
model.y = ...

# Parámetros
model.P = ...
model.VitA = ...
model.VitB = ...
model.VitA_UB = ...
model.VitB_LB = ...
model.BigM = ...

### Funcion objetivo
$$
\begin{align}
\text{Coste total} & = \sum_{c\in C} x_{c} P_{c} \nonumber
\end{align}
$$

In [ ]:
model.obj = ...

### Restricciones estructurales
- Balance de materia
$
\begin{align}
\sum_{c\in C} x_{c} & = 1 \nonumber
\end{align}
$

In [ ]:
model.massfraction = ...

- Especificaciones del producto final
$
\begin{align}
\sum_{c\in C} x_{c}VitA_{c} & \leq VitA^{UB} \nonumber \\
\sum_{c\in C} x_{c}VitB_{c} & \geq VitB^{LB} \nonumber
\end{align}
$

In [ ]:
model.comp_ub = ...
model.comp_lb = ...

# Restricciones lógicas: incompatibilidad de componentes

- Reformulación Big-M (manual)
$
\begin{align}
x_A & \leq M\;y \nonumber \\
x_B & \leq (1-M)\;y \nonumber
\end{align}
$


In [ ]:
model.ref_BigM = ConstraintList()
for pair in model.C_incompat:
    a, b = pair
    model.ref_BigM.add(model.x[a] <= model.BigM*model.y[pair])
    model.ref_BigM.add(model.x[b] <= model.BigM*(1-model.y[pair]))        

- Disyunciones

Antes de utilizar la enxtensión para GDP de Pyomo, hay que importar las funciones correspondientes. Para modelar la siguiente disyunción, se necesitarán los comandos `Disjunct`, `Disjunction` y, más adelante, `TransformationFactory`.

$$
\left[
\begin{array}{c}
Y \\
x_B=0
\end{array}
\right] 
\lor 
\left[
\begin{array}{c}
\neg Y \\
x_A=0
\end{array}
\right]
$$

In [ ]:
# Importar funciones necesarias
from pyomo.gdp import Disjunct, Disjunction

# Término que se cumple si Y = True

# Término que se cumple si Y = False

# Añadir restricción en forma de disyunción


## Resolución
### Reformulación Big M manual

In [ ]:
# Desactivar restricciones y mantener activa solo la que se ha definido manualmente (model.ref_BigM)
model.term1.deactivate()
model.term2.deactivate()
model.disjunction.deactivate()

# Llamada al solver
opt = SolverFactory('glpk')
results = opt.solve(model)

# Inspeccionar la solución
for c in model.C:
    print(f"{c} = {model.x[c]()}")

### Extensión Pyomo GDP
Aplica el método de la envolvente convexa para reformular la disyunción que aparece en este problema. La sintaxis que debes utilizar es `TransoformationFactory(trf_name).apply_to(model_name)`.

In [ ]:
# Acivar restricciones term1, term2, disjunction
model.term1.activate()
model.term2.activate()
model.disjunction.activate()

# Desactiva la restriccion ref_BigM 
model.ref_BigM.deactivate()

# Aplicar transformación deseada ("hull" o "bigm")
transformation = 'hull'
if transformation == 'bigm':
    TransformationFactory('gdp.bigm').apply_to(model)
elif transformation == 'hull':
    TransformationFactory('gdp.hull').apply_to(model)
  
# Llamada al solver
opt = SolverFactory('glpk')
results = opt.solve(model)

# Inspección de la solución
for c in model.C:
    print(f"{c} = {model.x[c]()}")